<span style="font-size: 14pt">MIPT, Advanced ML, Autumn 2017</span>

<span style="font-size: 16pt"> HW #9: Recommendation System

<span style="color:blue; font-size: 12pt">Andrey Saitgalin </span>
<span style="color:blue; font-size: 12pt; font-family: 'Verdana'">andrey.saitgalin@gmail.com</span>, 

<span style="color:blue; font-size: 12pt">Alexey Dral </span>,
<span style="color:blue; font-size: 12pt; font-family: 'Verdana'">aadral@gmail.com</span>

<h1 align="center">Organization Info</h1> 

**Дополнительный материал для выполнения дз**:
<a href= "https://github.com/ml-mipt/ml-mipt-part2/tree/master/2017/lectures">Лекции по курсу</a>

**Оформление дз**: 
- Присылайте выполненное задание на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2017_Aut_fall <номер_группы> <фамилия>``, к примеру -- ``ML2017_Aut_fall 401 ivanov``
- Выполненное дз сохраните в файл ``<фамилия>_<группа>_task<номер>.ipnb``, к примеру -- ``ivanov_491_task9.ipnb``

**Вопросы**:
- Присылайте вопросы на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2017_Aut_fall Question <Содержание вопроса>``

** Дедлайн**:
<span style="color:red; font-size: 12pt">16.12.2017 23:59 </span>

--------
- **PS1**: Мы используем автоматические фильтры, и просто не найдем ваше дз, если вы не аккуратно его подпишите.
- **PS2**: Напоминаем, что дедлайны жесткие, письма пришедшие после автоматически удаляются =( чтобы соблазна не было.

<h1 align="center">Check Questions (0%)</h1> 
Вопросы отсутствуют

<h1 align="center"> Practical tasks (100%)</h1>


# Построение content-based рекомендательной системы
## Part 1
Для каждого фильма получите описание в текстовом виде с IMDB или (и) The movie database (у них есть открытое API). https://www.themoviedb.org/?language=ru. Вывести описание фильма под id 778. Также можно использовать рецензии.

In [1]:
import tmdbsimple as tmdb
tmdb.API_KEY = open('api_key.txt').readline().rstrip();

In [2]:
tmdb.API_KEY[:5]

'6bc8a'

In [3]:
tmdb.Movies(778).info()

{'adult': False,
 'backdrop_path': '/zF95knRgouUrBNbUh5oqRI5NIcE.jpg',
 'belongs_to_collection': None,
 'budget': 0,
 'genres': [{'id': 35, 'name': 'Comedy'}],
 'homepage': None,
 'id': 778,
 'imdb_id': 'tt0046487',
 'original_language': 'fr',
 'original_title': 'Les vacances de Monsieur Hulot',
 'overview': 'Monsieur Hulot, Jacques Tati’s endearing clown, takes a holiday at a seaside resort, where his presence provokes one catastrophe after another. Tati’s masterpiece of gentle slapstick is a series of effortlessly well-choreographed sight gags involving dogs, boats, and firecrackers; it was the first entry in the Hulot series and the film that launched its maker to international stardom.',
 'popularity': 5.056227,
 'poster_path': '/r4F4tsU0Ajeh9ZYUkWOJSYmioj7.jpg',
 'production_companies': [{'id': 478,
   'logo_path': None,
   'name': 'Cady',
   'origin_country': ''},
  {'id': 479, 'logo_path': None, 'name': 'Discina', 'origin_country': ''}],
 'production_countries': [{'iso_3166_1': 

In [4]:
from tqdm import tqdm

In [5]:
n = 100
infos = dict()
for i in tqdm(range(n)):
    try:
        movie = tmdb.Movies(i)
        infos[i] = movie.info()
    except:
        pass

100%|██████████| 100/100 [01:27<00:00,  1.14it/s]


In [6]:
overviews = { key: infos[key]['overview'] for key in infos.keys()}
list(overviews.items())[-2:]

[(98,
  "In the year 180, the death of emperor Marcus Aurelius throws the Roman Empire into chaos.  Maximus is one of the Roman army's most capable and trusted generals and a key advisor to the emperor.  As Marcus' devious son Commodus ascends to the throne, Maximus is set to be executed.  He escapes, but is captured by slave traders.  Renamed Spaniard and forced to become a gladiator, Maximus must battle to the death with other men for the amusement of paying audiences."),
 (99,
  "A single mother in Madrid sees her only son die on his birthday as he runs to seek an actress' autograph. Beside herself with grief, she returns to Barcelona to tell the boy's father about the death of the son he never knew he had.")]

## Part 2 
Извлеките текстовые признаки у фильмов: tf idf, биграммы (может какие-то еще). Биграммы - статистика по биграммам, сколько какая биграмма встречается. Формулы для TF-IDF смотреть тут: https://en.wikipedia.org/wiki/Tf%E2%80%93idf 
    1. Используйте TF с double 0.5 normalization
    2. Используйте стандартный IDF: 
$$idf(t, D) = \frac{log N}{(1 + |d in D : t in d|)}$$

In [7]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), analyzer='word')
bigram_vectorizer_for_tf_idf = CountVectorizer(ngram_range=(1, 1))
X = bigram_vectorizer.fit_transform(list(overviews.values()))
X_tf_idf = bigram_vectorizer_for_tf_idf.fit_transform(list(overviews.values()))

In [8]:
print(type(X_tf_idf))
print(type(X))

<class 'scipy.sparse.csr.csr_matrix'>
<class 'scipy.sparse.csr.csr_matrix'>


In [9]:
import numpy as np

In [10]:
# index to id list
ix_id = np.sort(list(overviews.keys()))
# id to index dict
id_ix = {ix_id[ix]:ix for ix in range(len(ix_id))}

In [11]:
def tf(t,d):
    return 0.5 + 0.5 * X_tf_idf[id_ix[d], t] / max(X_tf_idf[id_ix[d], :].data)

def idf(t,D):
    return np.log(len(D) / len(X_tf_idf[:, t].indices))

def tf_idf(t,d,D):
    return tf(t,d) * idf(t,D)

In [12]:
from scipy.sparse import csr_matrix

In [13]:
data, row_ix, col_ix = [], [], []

for d in ix_id:
    for t in X_tf_idf[id_ix[d], :].indices:
        data.append(tf_idf(t, d, ix_id))
        row_ix.append(id_ix[d])
        col_ix.append(t)

tf_idf_matrix = csr_matrix((data, (row_ix, col_ix)))

## Part 3 
Выведите подсчитанные признаки из пункта 2 для фильма с id 99114.

Cделаем для фильма с id=98

In [14]:
print(X[id_ix[99], :].data)
print(tf_idf_matrix[id_ix[99], :].data)
print(overviews[99])

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 3 1 3 1 3 1]
[1.5157317  2.81300514 0.81585029 0.92419624 2.81300514 2.81300514
 2.81300514 2.81300514 1.8888089  1.74004653 2.81300514 1.5157317
 2.81300514 2.81300514 1.00063188 1.1564007  2.08059694 0.54554022
 0.29021205 2.81300514 2.81300514 1.8888089  2.81300514 0.21845827
 0.85004582 1.8888089  2.81300514 2.81300514 2.81300514 2.81300514
 1.8888089  2.81300514 2.02312353 2.81300514 0.19415601 0.32768741
 0.69430258]
A single mother in Madrid sees her only son die on his birthday as he runs to seek an actress' autograph. Beside herself with grief, she returns to Barcelona to tell the boy's father about the death of the son he never knew he had.
